In [79]:
import pandas as pd

df = pd.read_csv("reddit_wsb.csv")

df = df.dropna(subset=['body'])

df['document'] = (df['title'].fillna('') + ' ' + df['body']).str.strip()

df = df.reset_index(drop=True)

documents = df['document'].tolist()

print(f"Loaded {len(documents)} documents")

Loaded 24738 documents


In [80]:
import re

def normalize(text):
    text = text.lower()                        # lowercase
    text = re.sub(r'[^a-z0-9\s]', ' ', text)   # remove punctuation/symbols
    text = re.sub(r'\s+', ' ', text).strip()   # collapse spaces
    return text

documents = [normalize(doc) for doc in documents]

sample_100 = documents[:100]
sample_1000 = documents[:1000]
sample_10000 = documents[:10000]

# Sample Normalized Document
print(documents[0])

exit the system the ceo of nasdaq pushed to halt trading to give investors a chance to recalibrate their positions https mobile twitter com mediaite status 1354504710695362563 https mobile twitter com mediaite status 1354504710695362563 now sec is investigating brokers are disallowing buying more calls this is the institutions flat out admitting they will change the rules to bail out the rich but if it happens to us we get a well shucks you should have known investing is risky have you tried cutting out avocados and coffee maybe doing uber on the side we may have collectively driven up enough sentiment in wall street to make other big players go long on gme with us we do not have the money to move the stock as much as it did alone we didn t hurt wall street as a whole just a few funds went down while others went up and profited off the shorts the same as us the media wants to pin the blame on us it should be crystal clear that this is a rigged game by now its time to build new exchange

In [81]:
import random
import hashlib

def shingle (text, k):
    """Generate k-shingles from the input text.

    Args:
        text (str): The input text to generate shingles from.
        k (int): The length of each shingle.

    Returns:
        set: A set of k-shingles.
    """
    shingles = set()
    text_length = len(text)
    for i in range(text_length - k + 1):
        shingle = text[i:i + k]
        shingles.add(shingle)
    return shingles

def stable_hash64(s: str) -> int:
    """Deterministic 64-bit integer hash from a string using SHA-1 (first 8 bytes)."""
    h = hashlib.sha1(s.encode('utf-8')).digest()
    return int.from_bytes(h[:8], 'big')

def hashShingle (text, k):
    """Generate hashed k-shingles from the input text.

    Args:
        text (str): The input text to generate hashed shingles from.
        k (int): The length of each shingle.

    Returns:
        set: A set of hashed k-shingles.
    """
    shingles = shingle(text, k)
    hashed_shingles = set()
    for sh in shingles:
        hashed_shingles.add(stable_hash64(sh))
    return sorted(hashed_shingles)

shingle_sets = {}
for i, doc in enumerate(documents, start=1):
    shingles = hashShingle(doc, 5)
    if shingles:  # non-empty set
        shingle_sets[f"doc{i}"] = shingles


In [82]:
sample100_shingle_sets = {}
for i, doc in enumerate(sample_100, start=1):
    shingles = hashShingle(doc, 5)
    if shingles:  # non-empty set
        sample100_shingle_sets[f"doc{i}"] = shingles

sample1000_shingle_sets = {}
for i, doc in enumerate(sample_1000, start=1):
    shingles = hashShingle(doc, 5)
    if shingles:  # non-empty set
        sample1000_shingle_sets[f"doc{i}"] = shingles

sample10000_shingle_sets = {}
for i, doc in enumerate(sample_10000, start=1):     
    shingles = hashShingle(doc, 5)
    if shingles:  # non-empty set
        sample10000_shingle_sets[f"doc{i}"] = shingles

In [83]:
def jaccardSimilarity (j1, j2):
    """Calculate the Jaccard similarity between two sets.

    Args:
        set1 (set): The first set.
        set2 (set): The second set.

    Returns:
        float: The Jaccard similarity between the two sets.
    """
    set1 = set(j1)
    set2 = set(j2)
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    if not union:
        return 0.0
    return len(intersection) / len(union)

In [84]:
import numpy as np

p = 2**61 - 1
num_hashes = 100

a = np.random.randint(1, p, size=num_hashes, dtype=np.uint64)
b = np.random.randint(0, p, size=num_hashes, dtype=np.uint64)

def minHash_vectorized(shingle_set):
    # Convert shingle_set to a NumPy array for vector math
    x = np.array(list(shingle_set), dtype=np.uint64)

    # Broadcasted hashing:
    # For each hash function (row), apply (a*x + b) % p to all shingles
    hashes = (a[:, None] * x[None, :] + b[:, None]) % p

    # Take the minimum per row (i.e., per hash function)
    signatures = np.min(hashes, axis=1)

    return signatures.tolist()

# Apply to all documents
minhash_signatures = {k: minHash_vectorized(v) for k, v in shingle_sets.items()}

print(minhash_signatures["doc1"])  # Example output of minhash signature for document 1

[345089397602898, 2567974316486846, 3080072189639001, 3621578707448673, 620606878809477, 4322113570144494, 2372926116229249, 290955566547970, 1709157416065581, 2223315077028049, 248004311737834, 1632258557387183, 335126648437888, 7086949424797277, 194653819878190, 6048330392162801, 2513145045145986, 564396962657371, 183101369960174, 1906890662224196, 4217464263895757, 4667167192987965, 569015621626730, 358849543945644, 442077367808197, 1296410657099288, 416772951070937, 541889611012223, 1886254666446940, 408566471869421, 108848889045944, 7113398561100318, 234624603408151, 2483608996832255, 1300276538962330, 2269405630379042, 3614216203741224, 5476562412989528, 1106574343516144, 6085346997284083, 2495858697633621, 3461560054030472, 5931995726880241, 5535099076886260, 1823332974900962, 23075505049288, 2036555750494537, 248740859180053, 585931816025459, 3425528873223348, 600033553340304, 814567622128715, 164706312991875, 590610406884094, 1655171496247450, 2551147844553372, 103264893337502

In [85]:
def compareSignatures (m1, m2):
    agree = 0
    for i in range(len(m1)):
        if m1[i] == m2[i]:
            agree += 1
    return agree / len(m1)


In [86]:
from collections import defaultdict
from itertools import combinations
import numpy as np

def lsh_candidates(signatures, bands, rows_per_band):
    doc_ids = list(signatures.keys())
    sig_matrix = np.array(list(signatures.values()), dtype=np.uint64)
    assert sig_matrix.shape[1] == bands * rows_per_band

    buckets = [defaultdict(list) for _ in range(bands)]

    # Precompute slices for all bands
    for band in range(bands):
        start = band * rows_per_band
        end = start + rows_per_band
        band_slice = sig_matrix[:, start:end]

        # Efficient string join instead of str(tuple(...))
        # Much faster, same deterministic content
        for i, row in enumerate(band_slice):
            s = ','.join(map(str, row))
            bucket_hash = stable_hash64(s)
            buckets[band][bucket_hash].append(doc_ids[i])

    # Generate candidate pairs
    candidates = set()
    for band_dict in buckets:
        for doc_list in band_dict.values():
            if len(doc_list) > 1:
                doc_list = sorted(set(doc_list))
                candidates.update(combinations(doc_list, 2))

    return candidates


In [93]:
def compare_candidates(signatures, candidate_pairs, threshold=0.8, verbose=False):
    """
    Compare candidate pairs efficiently and filter by similarity threshold.
    """
    if verbose:
        print("\n=== STEP 5: OFFICIAL CANDIDATE COMPARISON ===")
        print(f"Similarity threshold: {threshold}\n")

    results = {}

    # Skip sorting (saves memory)
    for i, (d1, d2) in enumerate(candidate_pairs):
        sig1, sig2 = signatures[d1], signatures[d2]
        similarity = compareSignatures(sig1, sig2)

        if similarity >= threshold:
            results[(d1, d2)] = similarity

        # Lightweight progress print
        if verbose and i % 1000 == 0:
            print(f"Processed {i} pairs... current matches: {len(results)}")

    if verbose:
        print(f"\nTotal qualifying pairs: {len(results)}")

    return results



signatures = minhash_signatures

# Calculates candidate pairs based on LSH
# Messing around with bands and rows_per_band heavily changes the number of candidates
candidates = lsh_candidates(signatures, bands=10, rows_per_band=10)

# Compares candidate pairs and accepts only those above the threshold
similar_pairs = compare_candidates(signatures, candidates, threshold=0.8, verbose=True)

print(len(candidates))
print(len(similar_pairs))




=== STEP 5: OFFICIAL CANDIDATE COMPARISON ===
Similarity threshold: 0.8

Processed 0 pairs... current matches: 1
Processed 1000 pairs... current matches: 832
Processed 2000 pairs... current matches: 1697
Processed 3000 pairs... current matches: 2556
Processed 4000 pairs... current matches: 3420
Processed 5000 pairs... current matches: 4290
Processed 6000 pairs... current matches: 5141
Processed 7000 pairs... current matches: 5980
Processed 8000 pairs... current matches: 6817
Processed 9000 pairs... current matches: 7670
Processed 10000 pairs... current matches: 8530
Processed 11000 pairs... current matches: 9384
Processed 12000 pairs... current matches: 10220
Processed 13000 pairs... current matches: 11034
Processed 14000 pairs... current matches: 11884
Processed 15000 pairs... current matches: 12714
Processed 16000 pairs... current matches: 13528
Processed 17000 pairs... current matches: 14362
Processed 18000 pairs... current matches: 15224
Processed 19000 pairs... current matches: 1

In [64]:
sample100_minhash_signatures = {k: minHash_vectorized(v) for k, v in sample100_shingle_sets.items()}

In [71]:
sample100_signature = sample100_minhash_signatures

sample100_candidates = lsh_candidates(sample100_signature, bands=25, rows_per_band=4)

similar_pairs_100 = compare_candidates(sample100_signature, sample100_candidates, threshold=0.8, verbose=True)

print(len(similar_pairs_100))


=== STEP 5: OFFICIAL CANDIDATE COMPARISON ===
Similarity threshold: 0.8

Processed 0 pairs... current matches: 0

Total qualifying pairs: 1
1


In [ ]:
#Brute force Jaccard similarity 

results = {}
for i in range(len(sample_100)):
    print(i)
    doc1 = sample_100[i]
    for j in range(i + 1, len(sample_100)):
        doc2 = sample_100[j]
        similarity = jaccardSimilarity(doc1, doc2)
        if (similarity >= 0.8):
            results[(i, j)] = similarity
print(len(results))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
2745


In [31]:
# 1) How many docs total and how many unique signatures?
num_docs = len(signatures)
unique_sigs = len({tuple(sig) for sig in signatures.values()})
print("docs:", num_docs, "unique signatures:", unique_sigs,
      f"({unique_sigs/num_docs:.2%} unique)")

# 2) Are any signatures literally identical objects? (accidental list * n bug)
same_object_count = len([1 for s in signatures.values() if id(s) == id(next(iter(signatures.values())))])
print("Example identical-object check (should be 1):", same_object_count)

# 3) Check signature length and a quick per-position entropy-ish check
sig_len = len(next(iter(signatures.values())))
print("signature length:", sig_len)

# per-position distinct counts (spot collisions)
from collections import Counter
pos_counters = [Counter() for _ in range(sig_len)]
for sig in signatures.values():
    for i, v in enumerate(sig):
        pos_counters[i][v] += 1
distinct_counts = [len(c) for c in pos_counters]
print("distinct values per position (first 10):", distinct_counts[:10])
print("min/median/max distinct per position:", min(distinct_counts), sorted(distinct_counts)[len(distinct_counts)//2], max(distinct_counts))

# 4) Sample pairwise similarities (random small sample)
import random
pairs = []
docs = list(signatures.keys())
for _ in range(1000):
    a,b = random.sample(docs, 2)
    sigA, sigB = signatures[a], signatures[b]
    matches = sum(1 for x,y in zip(sigA,sigB) if x==y)
    pairs.append(matches/len(sigA))
import statistics
print("sample similarity: mean", statistics.mean(pairs), "median", statistics.median(pairs),
      "90th pct", sorted(pairs)[int(.9*len(pairs))])


import statistics

sims = list(similar_pairs.values())
print("min", min(sims), "median", statistics.median(sims),
      "max", max(sims))


docs: 24730 unique signatures: 24445 (98.85% unique)
Example identical-object check (should be 1): 1
signature length: 100
distinct values per position (first 10): [2580, 2601, 2965, 2721, 2531, 2830, 2316, 2365, 2564, 2704]
min/median/max distinct per position: 1796 2584 3066
sample similarity: mean 0.029580000000000002 median 0.02 90th pct 0.07
min 0.81 median 0.97 max 1.0


**Introduction**

In this project, we implemented the neccessary functionality for efficiently combing through a dataset of text documents, and being able to identify and record pairs of documents that are similar to each other. Brute force checking all possible pairs in a dataset to try and identify similar pairs is not a viable strategy, as the n(n-1)/2 comparisons needed is O(n^2) and renders this method obsolete on measurably large datasets. The process that we implemented was to first shingle our data, before then creating a min hash signature matrix for our dataset, and then running locality sensitive hashing (LSH) on our signature matrix to be able to identify candidate pairs that are highly likely to be similar above a certain threshold. This process allowed for a much more time efficient method of identifying similar pairs throughout a dataset of documents over standard naive pair checking.

**Instructions**

++ How to run our code and how to be able to pull in our dataset

**Shingling**

The first step that needed to be done was to shingle our data to be able to identify the parts that make up each of our documents, in order to use those parts as the basis of our document comparison. Shingling allows us to identify all letter sequences of a certain amount that appear in our document, and to use that as the contents of the document to compare to others. Once the shingles of a document are created, each shingle in that document's shingle set is then hashed to allow for improved computational cost, as they can now be represented and compared as numbers rather than as more memory intensive strings, and allows us to preform min-hashing afterwards. Given that our documents were generally around a medium length (reddit posts), we chose the shingle size for our dataset to be 5. When shingling our full dataset (24738 documents), it takes around 20-22 seconds to hash shingle all the documents.

**Min-Hashing**

Once we have the hashed shingles of each document, we then go about min-hashing each shingle set to obtain the signautre vector for each document, which when put together forms the signature matrix of our dataset. Min-hashing allows us to represent our longer hashed shingle sets, into much shorter representative "signatures" of the shingle set, and this allows for much less expensive comparisons between two documents, as we do not need to compute the intersection and union of large sets, and can compare the much shorter signatures instead. Our min-hash works by creating 100 independent hash functions, and having each hash function be applied to each value in the shingle set. 

For each hash function, the lowest value that was computed across all the hashes is recorded, and the min hash vector for a document is comprised of the 100 "lowest values" that each hash function computed. With this, the larger shingle set for each document can be represented by its much smaller 100 value signature vector. The signature vectors for each document are then combined to create the signature matrix of our dataset, which can be used for Locality Snesitive Hashing to be able to identify candidate pairs that might be similar. When min-hashing all of our documents, it takes around 15 seconds to be able to create the signature matrix.

**Locality Sensitive Hashing**

The final step in our implemented process is to do locality sensitive hashing onto our signature matrix. Locality sensitive hashing splits up the matrix into "bands" of rows, where in each band, the columns of our matrix within that band get hashed into buckets. Once this hashing is done across all bands, we can then identify "candidate pairs" as any pair of documents where in at least one band, they were hashed to the same bucket (in other words, any two documents where the signature vectors are identical in at least one section of rows). These candidate pairs are then the pairs that actually get compared to then confirm whether or not they are actually similar. LSH significantly reduces how many similarity comparisons we need to do by filtering out the pairs that do not have an exact signature match in any band, and allows us to be able to sift through similar documents on much larger datasets as a result. 

The band size plays a large factor in LSH, as a larger number of rows per bands (i.e. less bands) causes for less false positives but more false negatives (as now for a pair to be a candidate pair, they must agree exactly over a larger sequence of rows). As a result of this, we tested a few different configurations of band sizes to see which were better suited to find a good balance between low false-positives, and low false-negatives. The different band sizes also caused some different times for how long the LSH took, as with less bands, because the critertia for being a candidate pair is more strict, there are less candidate pairs identified and thus less comparisons needed. For our full dataset, the time it took for LSH to identify similar pairs was from around 1-3.5 seconds depending on our band size.

**Testing**

The overwehlming benefit of min-hashing/LSH is that it allows for much faster similarity comparisons, and narrows the field of pairs to search through significantly, which allows for much faster run times over a brute-force approach of checking the Jaccard Similarity across all pairs of documents. We set up samples of 100, 1000, 10000, and all of our documents (24738), and identified similar pairs using the brute force method and with the min-hashing + LSH process to compare the time it took to complete processing. We also tested on an LSH with 4 bands, 10 bands, and 20 bands (25, 10, and 5 rows per band respectively) to obersve how much decreasing the band size would increase the processing time:

|             |  100     |  1000  |  10000  |  All    |
|:-----------:|:--------:|:------:|:-------:|:-------:|
| **Naïve**   | .2 sec   | 5.4 sec| 779 sec | N/A     |
| **4 bands** | < .1 sec | .1 sec | 1.6 sec | 16.7 sec|
| **10 bands**| < .1 sec | .6 sec | 4.9 sec | 17.5 sec|
| **20 bands**| .1 sec   | 1.0 sec| 7.6 sec | 19.2 sec|

The Naive Jaccard method works fine on very small datasets, but given the O(n^2) nature of this approach, the computing time becomes very large as the dataset size increases, going from one fifth of a second to search through 100 documents, to almost 13 minutes for a dataset of 10000. With this method, searching through our whole dataset of nearly 25000 documents to find similar pairs is simply not feasible, and so we can see plainly the benefit of Min-hashing + LSH, as we are able to search for similar pairs through our whole dataset in a reasonable timeframe (16-20 sec). We can also see that slight difference mentioned before in the time the Min-Hashing + LSH approach takes depending on the band size, as for smaller bands sizes (more bands) it takes more time during the LSH portion as more pairs get searched through due to the less strict requirements for being identified as a candidate pair. 

Given that the band size has an impact on how many pairs are identified as being candidates for similarity, we also tested how many candidates pairs were identified with each of these band sizes, and how many false positives each band size ended up generating once the candidate pairs were checked to see whether they were actually similar:

|              | Candidate Pairs | Pairs Found  | False Positive % |
|:------------:|:---------------:|:------------:|:----------------:|
| **4 Bands**  |      12274      |     12274    |        0         |
| **10 Bands** |      28982      |     24448    |      15.64       |
| **20 Bands** |      56200      |     25443    |      54.80       |

With 4 bands, and a very tight restriction on candidate pairs, we noticed that we did not receive any false positives, and that all candidate pairs turned out to meet our similarity threshold of 80 percent. As the number of bands increased and the band size decreased, we noticed a very significant jump in candidate pairs. Having 10 bands of 10 rows gave us well over double the identified candidate pairs, and almost double the amount of actually similar pairs. This did also come up with the apperance of a sizeable amount of false positives, with 15.64 percent of the candidate pairs not actually being similar. Increasing the band number to 20 (5 rows per band) once again increased the number of candidate pairs by around double the amount, however this time the actual similar pairs did not increase nearly as significantly, gaining slightly less than 1000 actual similar pairs. This resulted in a skyrocketed amount of false positives in this band cnfiguration, as over half of the identified candidate pairs did not end up meeting our similarity threshold, and so the small improvement in identified pairs relative to the spike in false positives renders this configuration not optimal for our dataset. 

**Conclusions**

While brute force Jaccard similarity does identify all pairs that meet a similarity threshold, and can work fine with very small datasets, it proves infeasible quickly. Through our min-hashing + LSH implementation, we can see in our test dataset the impact it has, allowing us to sift through our whole dataset to look for similar pairs of documents, something that was not reasonable with a brute forch approach. We also saw the importance of configuring the band amount in LSH to tailor to what is important when identifying candidate pairs. With our dataset, should ensuring that no identified pair turns out to not be similar is neccessary, than a configuration with 4 bands might work. If making sure that a good amount of the actual similar pairs get identified, with some room for false positives being acceptable, a different configuration of 10 bands might be better suited, and this tuning of the LSH setup also applies outside of our dataset. 